# Size Transformers and Conductors Given Lengths

Objective
- Minimize cost.

Constraints
- Each distribution transformer has a maximum capacity in kVA.
- Each service drop must be within a certain percentage of the target voltage.

In [ ]:
# TODO: Use phase_count, motor_horsepower, target_in_volts to get current_in_amps
# TODO: Use phase_count, power_in_watts, target_in_volts to get current_in_amps
# TODO: Normalize calculation units to use base scientific units
# TODO: Convert table units to calculation units based on column name
# TODO: Check that input variables have required fields
# TODO: Match transformer secondary voltage to service drop voltage
# TODO: Support DC circuit calculations
# TODO: Estimate cost in waybill
# TODO: Add product URI to waybill as link for nickname
# TODO: Format waybill using markdown

In [ ]:
from os import getenv
from pathlib import Path

input_folder = Path(getenv('CROSSCOMPUTE_INPUT_FOLDER', 'batches/standard/input'))
output_folder = Path(getenv('CROSSCOMPUTE_OUTPUT_FOLDER', 'batches/standard/output'))
output_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
import json

with (input_folder / 'variables.dictionary').open('rt') as f:
    variables_dictionary = json.load(f)
variables_dictionary

In [ ]:
import pandas as pd

In [ ]:
maximum_voltage_drop_percent = variables_dictionary['maximum_voltage_drop_percent']
temperature_in_celsius = variables_dictionary['temperature_in_celsius']
# is_alternating_current = True

In [ ]:
transformer_types_table = pd.read_csv(input_folder / 'transformer-types.csv')
transformer_types_table.sort_values(['cost_in_dollars', 'size_in_kva'], inplace=True)
transformer_types_table[:2]

In [ ]:
conductor_types_table = pd.read_csv(input_folder / 'conductor-types.csv')
conductor_types_table.sort_values(['cost_in_dollars_per_foot', 'size_in_mm2'], inplace=True)
conductor_types_table[:2]

In [ ]:
with open(input_folder / 'service-drops.txt', 'rt') as f:
    service_drops_text = f.read().strip()
print(service_drops_text)

In [ ]:
service_drop_dictionaries = []
service_drop_dictionary = {}
for line in service_drops_text.splitlines():
    line = line.strip()
    if line.startswith('#'):
        continue
    if not line:
        if service_drop_dictionary:
            service_drop_dictionaries.append(service_drop_dictionary)
        service_drop_dictionary = {}
        continue
    key, value = line.split('=')
    key = key.strip()
    value = value.strip()
    try:
        float_value = float(value)
    except ValueError:
        pass
    else:
        try:
            integer_value = int(value)
        except ValueError:
            value = float_value
        else:
            value = integer_value if float_value == integer_value else float_value
    service_drop_dictionary[key] = value
if service_drop_dictionary:
    service_drop_dictionaries.append(service_drop_dictionary)    
service_drop_dictionaries

## Check that service drops do not exceed the maximum capacity of their transformer

In [ ]:
# Group service drops by transformer
from collections import defaultdict
service_drop_dictionaries_by_transformer_name = defaultdict(list)
for d in service_drop_dictionaries:
    service_drop_dictionaries_by_transformer_name[d['transformer_name']].append(d)
service_drop_dictionaries_by_transformer_name = dict(service_drop_dictionaries_by_transformer_name)    
service_drop_dictionaries_by_transformer_name

In [ ]:
transformer_dictionaries = []
for transformer_name in service_drop_dictionaries_by_transformer_name:
    minimum_power_in_kva = 0
    phase_counts = []
    service_drop_names = []
    for service_drop_dictionary in service_drop_dictionaries_by_transformer_name[transformer_name]:
        minimum_transformer_size_in_kva = service_drop_dictionary[
            'maximum_power_in_kilowatts'] / service_drop_dictionary['minimum_cosine_phi']
        minimum_power_in_kva += minimum_transformer_size_in_kva
        phase_counts.append(service_drop_dictionary['phase_count'])
        service_drop_names.append(service_drop_dictionary['service_drop_name'])
    phase_counts = set(phase_counts)        
    assert len(phase_counts) == 1
    phase_count = list(phase_counts)[0]
    transformer_type_dictionary = dict(transformer_types_table[(
        transformer_types_table['phase_count'] == phase_count
    ) & (
        transformer_types_table['size_in_kva'] >= minimum_power_in_kva
    )].iloc[0])
    transformer_dictionaries.append({
        'transformer_name': transformer_name,
        'transformer_type_nickname': transformer_type_dictionary['nickname'],
        'phase_count': phase_count,
        'minimum_size_in_kva': minimum_power_in_kva,
        'size_in_kva': transformer_type_dictionary['size_in_kva'],
        'service_drop_names': service_drop_names})
transformer_dictionaries

## Check that each service drop is within a certain percentage of the target voltage

In [ ]:
from infrastructure_resilience_toolkit.routines import compute_voltage_drop_percent

for service_drop_dictionary in service_drop_dictionaries:
    is_three_phase = service_drop_dictionary['phase_count'] == 3
    line_to_line_in_volts = service_drop_dictionary['line_to_line_in_volts']
    conductor_instalation_type = service_drop_dictionary['conductor_installation_type']
    conductor_length_in_feet = service_drop_dictionary['conductor_length_in_feet']
    conductor_spacing_in_inches = service_drop_dictionary['conductor_spacing_in_inches']
    current_in_amps = service_drop_dictionary['maximum_power_in_kilowatts'] / line_to_line_in_volts
    minimum_cosine_phi = service_drop_dictionary['minimum_cosine_phi']
    maximum_cosine_phi = service_drop_dictionary['maximum_cosine_phi']
    selected_conductor_types_table = conductor_types_table[(
        conductor_types_table['installation_type'] == conductor_instalation_type
    ) & (
        conductor_types_table['spacing_in_inches'] == conductor_spacing_in_inches
    ) & (        
        conductor_types_table['temperature_in_celsius'] == temperature_in_celsius
    )]
    for conductor_index, conductor_row in selected_conductor_types_table.iterrows():
        resistance_in_ohms = conductor_row['resistance_ac_in_ohms_per_kilofoot'] * conductor_length_in_feet / 1000
        reactance_in_ohms = conductor_row['reactance_inductive_in_ohm_per_kilofoot'] * conductor_length_in_feet / 1000
        ampacity_in_amps = conductor_row['ampacity_in_amps']
        voltage_drop_percents = []
        for cosine_phi in minimum_cosine_phi, maximum_cosine_phi:
            voltage_drop_percents.append(compute_voltage_drop_percent(
                is_three_phase=is_three_phase,
                source_line_to_line_in_volts=line_to_line_in_volts,
                current_in_amps=current_in_amps,
                resistance_in_ohms=resistance_in_ohms,
                reactance_in_ohms=reactance_in_ohms,
                cosine_phi=cosine_phi))
        voltage_drop_percent = max(voltage_drop_percents)
        if current_in_amps > ampacity_in_amps:
            continue
        if voltage_drop_percent <= maximum_voltage_drop_percent:
            break
    service_drop_dictionary.update({
        'conductor_type_nickname': conductor_row['nickname'],
        'size_in_awg': conductor_row['size_in_awg'],
        'size_in_kcmil': conductor_row['size_in_kcmil'],
        'size_in_mm2': conductor_row['size_in_mm2'],
        'ampacity_in_amps': ampacity_in_amps,
        'estimated_current_in_amps': current_in_amps,
        'estimated_voltage_drop_percent': voltage_drop_percent})
for d in service_drop_dictionaries:
    print(d)

## Save output files

In [ ]:
error_texts = []

In [ ]:
transformers_table = pd.DataFrame(transformer_dictionaries)
transformers_table.to_csv(output_folder / 'transformers.csv', index=False)

transformer_texts = []
for row_index, row in transformers_table.iterrows():
    transformer_name = row['transformer_name']
    transformer_lines = []
    transformer_lines.append('Transformer = %s' % transformer_name)
    transformer_lines.append('Minimum Size = %s kVA' % round(row['minimum_size_in_kva'], 2))
    if row['minimum_size_in_kva'] <= row['size_in_kva']:
        transformer_lines.append('Recommended Size = %s kVA' % row['size_in_kva'])    
        transformer_lines.append('**Recommended Transformer Type = %s**' % row['transformer_type_nickname'])
    else:
        transformer_lines.append('<span class="error">No Transformer Type Satisfies the Requirements</span>')
        error_texts.append(f'Transformer {transformer_name} power requirements exceed available transformer types.')
    transformer_texts.append('\n'.join(transformer_lines))
with open(output_folder / 'transformers.md', 'wt') as f:
    f.write('\n\n'.join(transformer_texts))

In [ ]:
conductors_table = pd.DataFrame(service_drop_dictionaries)
conductors_table.to_csv(output_folder / 'conductors.csv', index=False)

conductor_texts = []
for row_index, row in conductors_table.iterrows():
    conductor_name = 'From %s To %s' % (row['transformer_name'], row['service_drop_name'])
    conductor_lines = []
    conductor_lines.append(conductor_name)
    conductor_lines.append('Conductor Length = %s feet %s' % (
        row['conductor_length_in_feet'],
        row['conductor_installation_type']))
    conductor_lines.append('Maximum Power = %s kW' % row['maximum_power_in_kilowatts'])
    if row['estimated_voltage_drop_percent'] <= maximum_voltage_drop_percent:
        conductor_lines.append('**Recommended Conductor Type = %s**' % row['conductor_type_nickname'])
        conductor_lines.append('Estimated Voltage Drop = %s%% <= %s%%' % (
            round(row['estimated_voltage_drop_percent'], 5),
            maximum_voltage_drop_percent))
    else:
        conductor_lines.append('<span class="error">No Conductor Type Satisfies the Requirements</span>')
        error_texts.append(f'Conductor {conductor_name} power requirements exceed available conductor types.')
    conductor_texts.append('\n'.join(conductor_lines))
with open(output_folder / 'conductors.md', 'wt') as f:
    f.write('\n\n'.join(conductor_texts))

In [ ]:
transformers_waybill_table = pd.DataFrame(
    transformers_table['transformer_type_nickname'].value_counts()
).reset_index().rename(columns={
    'transformer_type_nickname': 'nickname',
    'count': 'count'})
transformers_waybill_table['unit'] = 'item'
transformers_waybill_table

In [ ]:
conductors_waybill_table = pd.DataFrame(
    conductors_table.groupby('conductor_type_nickname')['conductor_length_in_feet'].sum()
).reset_index().rename(columns={
    'conductor_type_nickname': 'nickname',
    'conductor_length_in_feet': 'count'})
conductors_waybill_table['unit'] = 'foot'
conductors_waybill_table

In [ ]:
waybill_table = pd.concat([
    transformers_waybill_table,
    conductors_waybill_table])
waybill_table.to_csv(output_folder / 'waybill.csv', index=False)
waybill_table.to_json(output_folder / 'waybill.json', orient='split', index=False)